In [2]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
import pandas as pd
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster

import utils
import matplotlib.pyplot as plt

from pandas import read_csv
import scipy.stats as stats
import joblib


['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\L

c:\Users\jz421\AppData\Local\anaconda3\envs\ieeg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


make model rdms here (just do congruency and switch type for testing for now 5/4)

In [3]:
# Define the conditions
conditions = ["i25s25", "i25s75", "i75s25", "i75s75", "i25r25", "i25r75", "i75r25", "i75r75",
              "c25s25", "c25s75", "c75s25", "c75s75", "c25r25", "c25r75", "c75r25", "c75r75"]

# Extract specific features from each condition
congruency = [cond[0] for cond in conditions]
switch_types = [cond[-3] for cond in conditions]
congruency_proportion = [int(cond[1:3]) for cond in conditions]
switch_proportion = [int(cond[-2:]) for cond in conditions]

# Number of conditions
n = len(conditions)

# initialize dict to store model RDMs
model_rdms = {'congruency': np.ones((n, n)), 'switchType': np.ones((n, n)),
              'congruencyProportion': np.ones((n, n)), 'switchProportion': np.ones((n, n))}

# Populate RDMs based on feature comparisons
for i in range(n):
    for j in range(n):
        if congruency[i] == congruency[j]:
            model_rdms['congruency'][i, j] = 0
        if switch_types[i] == switch_types[j]:
            model_rdms['switchType'][i, j] = 0
        if congruency_proportion[i] == congruency_proportion[j]:
            model_rdms['congruencyProportion'][i, j] = 0
        if switch_proportion[i] == switch_proportion[j]:
            model_rdms['switchProportion'][i, j] = 0

In [58]:
print(model_rdms['switchProportion'])

[[0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.]
 [1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.]
 [1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.]
 [1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.]
 [1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.]
 [1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.]
 [1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.]
 [1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.]
 [1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.]]


#### load in epochs data  
(get only acc trials, avg across epochs to make evoked using only acc trials, and then delete epochs from memory) 5/5

In [4]:
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']

make or load subjects electrodes to rois dict (mapping from electrode names to roi labels)

In [5]:
# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
filename = 'subjects_electrodestoROIs_dict.json'
subjects_electrodestoROIs_dict = utils.make_or_load_subjects_electrodes_to_rois_dict(filename, subjects)

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!


load accuracy arrays so we can filter by only accurate trials  

In [6]:
# this makes numpy arrays for each subject that are 0 or 1 for each trial based on accuracy
from makeRawBehavioralData import main
main()

# Directory where your .npy files are saved
npy_directory = r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\accArrays'  # Replace with your directory path if you're not Jim

acc_array = utils.load_acc_arrays(npy_directory, skip_subjects=['D107'])

load in behavioral data and convert block types to congruency and switch proportions

In [7]:
combined_data = pd.read_csv(r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\combinedData.csv')

# Apply the function to each row and create new columns
combined_data[['congruencyProportion', 'switchProportion']] = combined_data.apply(utils.map_block_type, axis=1)

load epochs and evoked for all 16 conditions

test these functions to just load in HG_ev1_rescaled cuz memory issues 5/6. Replace the ones in utils.py if these work.

In [8]:
def load_mne_objects(sub, output_name, task, just_HG_ev1_rescaled=False, LAB_root=None):
    """
    Load MNE objects for a given subject and output name, with an option to load only rescaled high gamma epochs.

    Parameters:
    - sub (str): Subject identifier.
    - output_name (str): Output name used in the file naming.
    - task (str): Task identifier.
    - just_HG_ev1_rescaled (bool): If True, only the rescaled high gamma epochs are loaded.
    - LAB_root (str, optional): Root directory for the lab. If None, it will be determined based on the OS.

    Returns:
    A dictionary containing loaded MNE objects.
    """

    # Determine LAB_root based on the operating system
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    # Get data layout
    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)

    # Ensure save directory exists
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Initialize the return dictionary
    mne_objects = {}

    if just_HG_ev1_rescaled:
        # Define path and load only the rescaled high gamma epochs
        HG_ev1_rescaled_file = f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif'
        HG_ev1_rescaled = mne.read_epochs(HG_ev1_rescaled_file)
        mne_objects['HG_ev1_rescaled'] = HG_ev1_rescaled
    else:
        # Define file paths
        HG_ev1_file = f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif'
        HG_base_file = f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif'
        HG_ev1_rescaled_file = f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif'
        
        # Load the objects
        HG_ev1 = mne.read_epochs(HG_ev1_file)
        HG_base = mne.read_epochs(HG_base_file)
        HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0))
        HG_ev1_rescaled = mne.read_epochs(HG_ev1_rescaled_file)
        HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

        mne_objects['HG_ev1'] = HG_ev1
        mne_objects['HG_base'] = HG_base
        mne_objects['HG_ev1_evoke'] = HG_ev1_evoke
        mne_objects['HG_ev1_rescaled'] = HG_ev1_rescaled
        mne_objects['HG_ev1_evoke_rescaled'] = HG_ev1_evoke_rescaled

    return mne_objects


def create_subjects_mne_objects_dict(subjects, output_names_conditions, task, combined_data, acc_array, just_HG_ev1_rescaled=False, LAB_root=None):
    """
    Adjusted to handle multiple conditions per output name, with multiple condition columns.

    Parameters:
    - subjects: List of subject IDs.
    - output_names_conditions: Dictionary where keys are output names and values are dictionaries
        of condition column names and their required values.
    - task: Task identifier.
    - combined_data: DataFrame with combined behavioral and trial information.
    - acc_array: dict of numpy arrays of 0 for incorrect and 1 for correct trials for each subject
    - LAB_root: Root directory for data (optional).
    """
    subjects_mne_objects = {}

    for sub in subjects:
        print(f"Loading data for subject: {sub}")
        sub_mne_objects = {}
        for output_name, conditions in output_names_conditions.items():
            print(f"  Loading output: {output_name} with conditions: {conditions}")
            
            # Build the filtering condition
            sub_without_zeroes = "D" + sub[1:].lstrip('0') 
            condition_filter = (combined_data['subject_ID'] == sub) # this previously indexed using sub_without_zeroes, but now just uses sub. 3/17.
                    
            for condition_column, condition_value in conditions.items():
                if isinstance(condition_value, list):
                    # If the condition needs to match any value in a list
                    condition_filter &= combined_data[condition_column].isin(condition_value)
                else:
                    # If the condition is a single value
                    condition_filter &= (combined_data[condition_column] == condition_value)
            
            # Filter combinedData for the specific subject and conditions
            subject_condition_data = combined_data[condition_filter]
            
            # Load MNE objects and update with accuracy data
            mne_objects = load_mne_objects(sub, output_name, task, just_HG_ev1_rescaled=just_HG_ev1_rescaled, LAB_root=None)
            
            if sub in acc_array:
                trial_counts = subject_condition_data['trialCount'].values.astype(int)
                accuracy_data = [acc_array[sub][i-1] for i in trial_counts if i-1 < len(acc_array[sub])] # Subtract 1 here for zero-based indexing in acc array.
                # Now pass trial_counts along with accuracy_data
                mne_objects['HG_ev1_rescaled'] = utils.add_accuracy_to_epochs(mne_objects['HG_ev1_rescaled'], accuracy_data)

            sub_mne_objects[output_name] = mne_objects
        subjects_mne_objects[sub] = sub_mne_objects

    return subjects_mne_objects

In [9]:
output_names = ["Stimulus_i25s25_fixationCrossBase_1sec_mirror_0to1Test", "Stimulus_i25s75_fixationCrossBase_1sec_mirror_0to1Test", 
                "Stimulus_i75s25_fixationCrossBase_1sec_mirror_0to1Test", "Stimulus_i75s75_fixationCrossBase_1sec_mirror_0to1Test", 
                "Stimulus_i25r25_fixationCrossBase_1sec_mirror_0to1Test", "Stimulus_i25r75_fixationCrossBase_1sec_mirror_0to1Test", 
                "Stimulus_i75r25_fixationCrossBase_1sec_mirror_0to1Test", "Stimulus_i75r75_fixationCrossBase_1sec_mirror_0to1Test", 
                "Stimulus_c25s25_fixationCrossBase_1sec_mirror_0to1Test", "Stimulus_c25s75_fixationCrossBase_1sec_mirror_0to1Test", 
                "Stimulus_c75s25_fixationCrossBase_1sec_mirror_0to1Test", "Stimulus_c75s75_fixationCrossBase_1sec_mirror_0to1Test",
                "Stimulus_c25r25_fixationCrossBase_1sec_mirror_0to1Test", "Stimulus_c25r75_fixationCrossBase_1sec_mirror_0to1Test",
                "Stimulus_c75r25_fixationCrossBase_1sec_mirror_0to1Test", "Stimulus_c75r75_fixationCrossBase_1sec_mirror_0to1Test",]
                
output_names_conditions = {
    "Stimulus_i25s25_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus_i25s75_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus_i75s25_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus_i75s75_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus_i25r25_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus_i25r75_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus_i75r25_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus_i75r75_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus_c25s25_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus_c25s75_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus_c75s25_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus_c75s75_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus_c25r25_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus_c25r75_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus_c75r25_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus_c75r75_fixationCrossBase_1sec_mirror_0to1Test": {
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    },                                                            
}

task='GlobalLocal'

# Assuming 'combined_data' is your DataFrame and 'subjects' is your list of subject IDs
subjects_mne_objects = create_subjects_mne_objects_dict(subjects, output_names_conditions, task="GlobalLocal", combined_data=combined_data, acc_array=acc_array, just_HG_ev1_rescaled=True)

# Save the subjects_mne_objects dictionary to a file
# joblib.dump(subjects_mne_objects, 'subjects_mne_objects.joblib')

Loading data for subject: D0057
  Loading output: Stimulus_i25s25_fixationCrossBase_1sec_mirror_0to1Test with conditions: {'congruency': 'i', 'congruencyProportion': '75%', 'switchType': 's', 'switchProportion': '25%'}
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_i25s25_fixationCrossBase_1sec_mirror_0to1Test_HG_ev1_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
7 matching events found
No baseline correction applied
0 projection items activated
Adding metadata with 0 columns
  Loading output: Stimulus_i25s75_fixationCrossBase_1sec_mirror_0to1Test with conditions: {'congruency': 'i', 'congruencyProportion': '75%', 'switchType': 's', 'switchProportion': '75%'}
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_i25s75_fixationCrossBase_1sec_mirror_0to1Test_H

In [10]:
# Load the subjects_mne_objects from the file. This is not equal to when we create subjects_mne_objects directly...test this later. 5/6.
# subjects_mne_objects_test = joblib.load('subjects_mne_objects.joblib')

load stimulus significant channels

In [11]:
sig_chans_per_subject = utils.get_sig_chans_per_subject(subjects, task='GlobalLocal', LAB_root=None)

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090
Loaded significant channels for subject D0094
Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103


get electrodes for each roi (definitions based on destrieux atlas)

In [12]:
rois_dict = {
    # 'dlpfc': ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    # 'acc': ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"],
    # 'parietal': ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"],
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'v1': ["G_oc-temp_med-Lingual", "S_calcarine", "G_cuneus"]
}

rois = list(rois_dict.keys())
electrodes_per_subject_roi, sig_electrodes_per_subject_roi = utils.make_sig_electrodes_per_subject_and_roi_dict(rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject)

For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI15', 'RAI16', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
For subject D0059, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF12', 'LPSF16']
For subject D0063, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LOF16', 'LASF10', 'LASF14', 'LASF15', 'LASF16', 'LM

get electrode counts for each roi (just for fun)

In [13]:
total_electrodes_info = utils.calculate_total_electrodes(sig_electrodes_per_subject_roi, electrodes_per_subject_roi)
for roi, counts in total_electrodes_info.items():
    print(f"Total number of significant {roi} electrodes across all subjects:", counts['total_significant_electrodes'])
    print(f"Total number of {roi} electrodes across all subjects:", counts['total_electrodes'])

Total number of significant lpfc electrodes across all subjects: 44
Total number of lpfc electrodes across all subjects: 105
Total number of significant v1 electrodes across all subjects: 7
Total number of v1 electrodes across all subjects: 17


check if any subjects have a different sampling rate than 2048 Hz

In [14]:
def check_sampling_rates(subjects_mne_objects):
    # This dictionary will store subjects with different sampling rates
    different_sampling_rates = {}
    
    # Iterate through each subject and their corresponding data
    for subject, data in subjects_mne_objects.items():
        # Access the specific Epochs object and check its sampling rate
        if 'Stimulus_i25s25_fixationCrossBase_1sec_mirror_0to1Test' in data:
            epochs = data['Stimulus_i25s25_fixationCrossBase_1sec_mirror_0to1Test']['HG_ev1_rescaled']
            sampling_rate = epochs.info['sfreq']
            
            # Check if the sampling rate is not 2048.0 Hz
            if sampling_rate != 2048.0:
                different_sampling_rates[subject] = sampling_rate
    
    return different_sampling_rates

# Assuming 'subjects_mne_objects' is your dictionary containing MNE objects for each subject
different_rates = check_sampling_rates(subjects_mne_objects)

# Print the results
if different_rates:
    print("Subjects with different sampling rates:")
    for subject, rate in different_rates.items():
        print(f"Subject {subject} has a sampling rate of {rate} Hz.")
else:
    print("All subjects have a sampling rate of 2048.0 Hz.")

All subjects have a sampling rate of 2048.0 Hz.


get trial averaged data

In [15]:
# Define time indices just for the function to work. Don't actually use time averaged data.
time_indices = {
    'firstHalfSecond': (2048, 3072),
    'secondHalfSecond': (3072, 4096),
    'fullSecond': (2048, 4096)
}
# Process the data
data_trialAvg_lists, data_trialStd_lists, _, overall_electrode_mapping, electrode_mapping_per_roi = utils.process_data_for_roi(
    subjects_mne_objects, output_names, rois, subjects, sig_electrodes_per_subject_roi, time_indices)

# need to figure out how to use the overall_electrode_mapping to get the electrode names back. Apparently I do this in the ANOVA in roi analysis. 5/5.

# Concatenate the data
concatenated_trialAvg_data = utils.concatenate_data(data_trialAvg_lists, rois, output_names)

Subject: D0057, ROI: lpfc, Num of Sig Electrodes: 2
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


c:\Users\jz421\Desktop\GlobalLocal\utils.py:269: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:270: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:269: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:270: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy(

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


c:\Users\jz421\Desktop\GlobalLocal\utils.py:269: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:270: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:269: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:270: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy(

...

c:\Users\jz421\Desktop\GlobalLocal\utils.py:563: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  epochs = subjects_mne_objects[sub][output_name]['HG_ev1_rescaled'].copy().pick_channels(sig_electrodes)
c:\Users\jz421\Desktop\GlobalLocal\utils.py:269: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:270: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:269: FutureWarning: The cur

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


c:\Users\jz421\Desktop\GlobalLocal\utils.py:563: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  epochs = subjects_mne_objects[sub][output_name]['HG_ev1_rescaled'].copy().pick_channels(sig_electrodes)
c:\Users\jz421\Desktop\GlobalLocal\utils.py:269: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:270: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:269: FutureWarning: The cur

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Subject: D0103, ROI: v1, Num of Sig Electrodes: 0


c:\Users\jz421\Desktop\GlobalLocal\utils.py:269: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:270: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
c:\Users\jz421\Desktop\GlobalLocal\utils.py:563: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  epochs = subjects_mne_objects[sub][output_name]['HG_ev1_rescaled'].copy().pick_channels(sig_electrodes)
c:\Users\jz421\Desktop\GlobalLocal\utils.py:269: FutureWarning: The cur

In [16]:
def compute_avg_std(data_dict):
    """
    Computes the mean and standard deviation across electrodes for each time point in each ROI and output name.

    Parameters:
    - data_dict (dict): A dictionary containing trial-averaged neural data (concatenated_trialAvg_data). It is expected to have a structure where each key is an output name,
      and its value is another dictionary. This nested dictionary should have ROIs as keys, and arrays of shape (n_electrodes, n_timepoints)
      as values.

    Returns:
    - tuple: A tuple containing two dictionaries:
        - mean_data: A dictionary with the same structure as input, but each array contains the mean across electrodes at each timepoint.
        - std_data: A dictionary with the same structure as input, but each array contains the standard deviation across electrodes at each timepoint.

    Example:
    mean_trialAvg_data, std_trialAvg_data = compute_avg_std(concatenated_trialAvg_data)
    print("Mean data for LPFC:", mean_trialAvg_data['Stimulus_i25s25_fixationCrossBase_1sec_mirror_0to1Test']['lpfc'])
    print("Standard deviation data for LPFC:", std_trialAvg_data['Stimulus_i25s25_fixationCrossBase_1sec_mirror_0to1Test']['lpfc'])
    """
    mean_data = {}
    std_data = {}
    for output_name, rois_data in data_dict.items():
        mean_data[output_name] = {}
        std_data[output_name] = {}
        for roi, data in rois_data.items():
            # Compute mean and std across the electrodes (axis=0)
            mean_data[output_name][roi] = np.mean(data, axis=0)
            std_data[output_name][roi] = np.std(data, axis=0)
    return mean_data, std_data


In [17]:
# get mean and std across electrodes for each time point, in the already trial-averaged data.
mean_across_elecs_trialAvg_data, std_across_elecs_trialAvg_data = compute_avg_std(concatenated_trialAvg_data)

In [54]:
mean_across_elecs_trialAvg_data

{'Stimulus_i25s25_fixationCrossBase_1sec_mirror_0to1Test': {'lpfc': array([-0.06955044, -0.07076683, -0.07198551, ...,  0.07580004,
          0.07368392,  0.07154889]),
  'v1': array([ 0.04172794,  0.0405265 ,  0.03925383, ..., -0.05908064,
         -0.0546062 , -0.05014162])},
 'Stimulus_i25s75_fixationCrossBase_1sec_mirror_0to1Test': {'lpfc': array([0.00353543, 0.00360855, 0.00372938, ..., 0.18935764, 0.18887467,
         0.18837217]),
  'v1': array([0.16014895, 0.16135009, 0.16254869, ..., 0.24513446, 0.24989514,
         0.25468031])},
 'Stimulus_i75s25_fixationCrossBase_1sec_mirror_0to1Test': {'lpfc': array([0.01571482, 0.01557537, 0.01540446, ..., 0.2058112 , 0.20614393,
         0.20650358]),
  'v1': array([ 0.20891506,  0.20568973,  0.20243682, ..., -0.08061834,
         -0.07925776, -0.07789581])},
 'Stimulus_i75s75_fixationCrossBase_1sec_mirror_0to1Test': {'lpfc': array([-0.07570329, -0.07525016, -0.07479778, ...,  0.10186556,
          0.10164739,  0.10146788]),
  'v1': arra

try kumar's suggestion of flattening the data to be more data driven 5/7  
this doesn't work yet..

In [44]:
def flatten_data_by_roi(data_dict):
    """
    Flattens the electrode data for each condition and ROI into one long vector per condition.

    Parameters:
    - data_dict: dict
        A dictionary where each key is a condition and each value is another dict with keys as ROIs
        and values as arrays of shape (n_electrodes, n_timepoints).

    Returns:
    - dict
        A dictionary where each key is an ROI and each value is a list of flattened data arrays from all conditions for that ROI.
    """
    roi_data = {}
    for condition, rois in data_dict.items():
        for roi, data in rois.items():
            flattened_data = data.reshape(-1)  # Flatten the electrode by time point data into one long vector
            if roi not in roi_data:
                roi_data[roi] = []
            roi_data[roi].append(flattened_data)
    return roi_data

In [53]:
concatenated_trialAvg_data

{'Stimulus_i25s25_fixationCrossBase_1sec_mirror_0to1Test': {'lpfc': array([[-0.27893369, -0.28428207, -0.2898255 , ...,  0.18689985,
           0.18496857,  0.18284192],
         [ 0.13412701,  0.13505303,  0.13582653, ...,  0.07705337,
           0.06740816,  0.05776003],
         [-0.02884331, -0.02668281, -0.02448124, ...,  1.00115065,
           1.00043964,  0.99929257],
         ...,
         [-0.64864258, -0.64674573, -0.64495685, ..., -0.33375768,
          -0.33872441, -0.34387202],
         [-0.26504986, -0.27681604, -0.28894591, ..., -0.49723368,
          -0.49161252, -0.48609994],
         [ 0.06897462,  0.07349734,  0.07797647, ...,  0.44334181,
           0.43653923,  0.42953495]]),
  'v1': array([[-1.00708318, -1.01879486, -1.03034383, ...,  0.53983793,
           0.55778205,  0.57542196],
         [ 0.32882339,  0.3298206 ,  0.33067294, ..., -0.80116273,
          -0.80160139, -0.80179718],
         [ 0.28468219,  0.28092601,  0.27710238, ...,  0.4442388 ,
           0.

In [51]:
flattened_trialAvg_data = flatten_data_by_roi(concatenated_trialAvg_data)

In [52]:
flattened_trialAvg_data['lpfc']

[array([-0.27893369, -0.28428207, -0.2898255 , ...,  0.44334181,
         0.43653923,  0.42953495]),
 array([-0.08951621, -0.08830033, -0.08691743, ...,  0.11797874,
         0.12544789,  0.13283763]),
 array([-0.25129884, -0.2467554 , -0.24234593, ...,  0.30658843,
         0.31322565,  0.3198349 ]),
 array([-0.02689337, -0.02567408, -0.02447877, ...,  0.23872906,
         0.24119757,  0.24352844]),
 array([-0.22088963, -0.21933408, -0.2176944 , ..., -0.07552303,
        -0.07817647, -0.08083785]),
 array([-0.1282848 , -0.10942167, -0.08952893, ...,  0.25782161,
         0.25897916,  0.26053852]),
 array([-0.0070959 , -0.00667725, -0.00610463, ...,  0.18140189,
         0.18130047,  0.18134487]),
 array([0.09793676, 0.09837488, 0.09883309, ..., 0.41638155, 0.4229201 ,
        0.42950953]),
 array([0.07465445, 0.0773366 , 0.07989213, ..., 0.01123999, 0.0117503 ,
        0.01229058]),
 array([-0.01969061, -0.0226547 , -0.02578209, ...,  0.2463944 ,
         0.24683187,  0.24731285]),
 a

#### Do RSA!  
1. Create time windows of 20 ms width, at 5 ms steps through the electrode and trial-averaged data.  
2. For each time window and ROI, calculate (1-Spearman correlation coefficient) between each condition. This will make a 16x16 neural RDM for each time window and ROI.  
3. For each neural RDM, find its correlation with each model RDM.

In [46]:
def create_time_windows(data, window_size_ms, step_size_ms, sampling_rate):
    """
    Generates a list of tuples representing the start and end samples for time windows.

    Parameters:
    - data: ndarray
        The electrode and trial averaged array from which to calculate the number of samples. Should just be a time series vector (mean_across_elecs_trialAvg_data).
    - window_size_ms: int
        Size of each time window in milliseconds.
    - step_size_ms: int
        The step size to move from one window to the next in milliseconds.
    - sampling_rate: float
        The sampling rate of the data in Hz.

    Returns:
    - list of tuples
        Each tuple contains the start and end sample indices for a time window.
    """
    window_size_samples = int(window_size_ms * sampling_rate / 1000)
    step_size_samples = int(step_size_ms * sampling_rate / 1000)
    total_samples = data.shape[0]
    windows = [(start, start + window_size_samples) for start in range(0, total_samples - window_size_samples + 1, step_size_samples)]
    return windows

def calculate_spearman_rdm(data, indices):
    """
    Calculates the Spearman rank correlation-based RDM for the given data and indices.

    Parameters:
    - data: list of ndarrays
        The condition data where each element is an array of time series data for a condition (after creating time windows from mean_across_elecs_trialAvg_data['output_name'])
    - indices: tuple
        The start and end indices for the time window to use in the correlation calculation.

    Returns:
    - ndarray
        A square array (RDM) where each element (i, j) is the Spearman correlation distance (1 - rho) between conditions i and j.
    """
    num_conditions = len(data)
    rdm = np.zeros((num_conditions, num_conditions))
    for i in range(num_conditions):
        for j in range(num_conditions):
            if i != j:
                correlation, _ = stats.spearmanr(data[i][indices[0]:indices[1]], data[j][indices[0]:indices[1]])
                rdm[i, j] = 1 - correlation
            else:
                rdm[i, j] = 0  # Diagonal elements are zero (perfect correlation with itself)
    return rdm

def make_neural_rdms(data_dict, sampling_rate=2048.0):
    """
    Makes a neural RDM across all conditions for each ROI and time window.

    Parameters:
    - data_dict: dict
        A dictionary where each key is a condition and each value is another dict with keys as ROIs
        and values as time series data arrays.
    - sampling_rate: int
        The sampling rate in the data, extracted from epochs.info['sfreq]. Should be 2048 Hz for all subjects for HG_ev1_rescaled.

    Returns:
    - dict
        A dictionary where each key is an ROI and each value is a list of tuples, each containing a sample window (40 samples wide, ~19.5 ms) and its corresponding RDM.
    """
    window_size_ms = 20
    step_size_ms = 5
    
    # Assuming all ROIs have the same time series length, use an arbitrary condition and ROI to determine the length of the time series
    example_condition = next(iter(data_dict.values()))
    example_data = next(iter(example_condition.values()))
    sample_windows = create_time_windows(example_data, window_size_ms, step_size_ms, sampling_rate)

    all_rdms = {}
    # First organize the data by ROI, aggregating across all conditions
    roi_data = {}
    for condition, rois in data_dict.items():
        for roi, data in rois.items():
            if roi not in roi_data:
                roi_data[roi] = []
            roi_data[roi].append(data)

    # Calculate RDMs for each ROI
    for roi, data in roi_data.items():
        all_rdms[roi] = []
        for window in sample_windows:
            rdm = calculate_spearman_rdm(data, window)
            all_rdms[roi].append((window, rdm))
    
    return all_rdms

# Make neural RDM comparing conditions for time windows of 40 samples (~20 ms), with steps of 10 samples (~5 ms). Keys are ROIs. Returns a tuple where 0th element is the sample window (tuple), and 1st element is the neural RDM (np array).
neural_rdms_by_samples_and_roi = make_neural_rdms(flattened_trialAvg_data, sampling_rate=2048.0)

AttributeError: 'list' object has no attribute 'values'

actually run rsa

In [59]:
def calculate_rsa(neural_rdms, model_rdms, rois):
    """
    Calculates the RSA between neural RDMs and model RDMs, using only the lower triangular part of the RDMs (excluding the diagonal).

    Parameters:
    - neural_rdms: dict
        A dictionary where keys are ROIs, each containing a list of tuples with sample windows and corresponding RDMs.
    - model_rdms: dict
        A dictionary containing the model RDMs with keys as model names.
    - rois: list
        List of ROIs to be analyzed.

    Returns:
    - dict
        A structured dictionary containing RSA results mapped by ROI, model type, and sample window.
    """
    rsa_results = {roi: {model_name: [] for model_name in model_rdms} for roi in rois}
    
    # Iterate over each ROI
    for roi in neural_rdms:
        for window, neural_rdm in neural_rdms[roi]:
            # Extract the lower triangular part of the neural RDM, excluding the diagonal
            tri_indices = np.tril_indices_from(neural_rdm, k=-1)
            neural_rdm_tril = neural_rdm[tri_indices]

            for model_name, model_rdm in model_rdms.items():
                # Extract the lower triangular part of the model RDM, excluding the diagonal
                model_rdm_tril = model_rdm[tri_indices]

                # Calculate Spearman correlation between the flattened lower triangular parts of each RDM
                correlation, _ = stats.spearmanr(neural_rdm_tril, model_rdm_tril)
                rsa_results[roi][model_name].append((window, correlation))
                
    return rsa_results

In [60]:
# Define ROIs based on keys in your neural_rdms_by_samples_and_roi (if they are consistent)
rois = list(neural_rdms_by_samples_and_roi.keys())

# Calculate RSA
rsa_results = calculate_rsa(neural_rdms_by_samples_and_roi, model_rdms, rois)

# Example: Print the RSA result for a specific ROI and model
print("RSA results for ROI 'lpfc' and model 'congruency':", rsa_results['lpfc']['congruency'])

RSA results for ROI 'lpfc' and model 'congruency': [((0, 40), 0.009713860608042253), ((10, 50), 0.010165768054877934), ((20, 60), 0.00838011243829298), ((30, 70), -0.018559596896870117), ((40, 80), -0.04115912423955696), ((50, 90), -0.054468062616419705), ((60, 100), -0.05742560447794475), ((70, 110), -0.058300000574168204), ((80, 120), -0.04259808778419234), ((90, 130), -0.03622338060534568), ((100, 140), 0.04246328874346146), ((110, 150), 0.025805432705378852), ((120, 160), -0.02416403449108552), ((130, 170), -0.05731177758744929), ((140, 180), -0.06605425213468732), ((150, 190), -0.05813223357404383), ((160, 200), -0.04808869239615399), ((170, 210), -0.04738509795531303), ((180, 220), -0.002416694267979027), ((190, 230), -0.02952913577845495), ((200, 240), 0.019861531515619292), ((210, 250), 0.07557070527894169), ((220, 260), 0.028655932485608097), ((230, 270), 0.025691525676752088), ((240, 280), -0.03406698613675259), ((250, 290), -0.07140819622708945), ((260, 300), -0.072472768926

plot congruency rsa for each roi

plot rsa results, different rois as different colors

In [61]:
def plot_rsa_correlation_by_time(rsa_results, model_name, rois, colors, sampling_rate=2048.0, time_shift=1000):
    """
    Plots RSA correlation over time for specified ROIs and model.

    Parameters:
    - rsa_results: dict
        A dictionary containing RSA results.
    - model_name: str
        The model name to plot RSA results for.
    - rois: list
        List of ROIs to be included in the plot.
    - colors: dict
        A dictionary mapping ROIs to plot colors.
    - sampling_rate: float, optional
        The sampling rate of the data in Hz. Default is 2048.0.
    - time_shift: int, optional
        Time in milliseconds to shift the x-axis. Default is 1000 ms.
    """
    # Create the figure and axis objects
    fig, ax = plt.subplots(figsize=(10, 6))

    # Loop over each ROI and its RSA results
    for roi in rois:
        if roi in rsa_results:
            times = []
            correlations = []
            for window, correlation in rsa_results[roi][model_name]:
                # Calculate the middle of the window
                middle_sample = (window[0] + window[1]) / 2
                time_ms = (middle_sample * 1000 / sampling_rate) - time_shift  # Subtract the shift to align times
                times.append(time_ms)
                correlations.append(correlation)

            # Sort the data by time for plotting lines
            sorted_indices = np.argsort(times)
            times = np.array(times)[sorted_indices]
            correlations = np.array(correlations)[sorted_indices]

            # Plot each ROI's data with a specific color and connect points with lines
            ax.scatter(times, correlations, color=colors[roi], label=f'{roi}')
            ax.plot(times, correlations, color=colors[roi])  # This line connects the dots

    # Adding labels and title
    ax.set_xlabel('Time from Stim Onset (ms)')
    ax.set_ylabel('Spearman Corr. Coeff.')
    ax.set_title(f'{model_name.capitalize()} RSA Correlation over Time by ROI')
    ax.legend()

    # Set x-axis limits to reflect the new time range
    ax.set_xlim(-1000, 1500)

    # Show the plot
    plt.show()

In [62]:
rois = ['lpfc', 'v1']
colors = {'lpfc': 'red', 'v1': 'blue'}

In [63]:
plot_rsa_correlation_by_time(rsa_results, 'congruency', rois, colors)

KeyboardInterrupt: 

switch type rsa

In [64]:
plot_rsa_correlation_by_time(rsa_results, 'switchType', rois, colors)

KeyboardInterrupt: 

congruency proportion rsa

In [65]:
plot_rsa_correlation_by_time(rsa_results, 'congruencyProportion', rois, colors)

KeyboardInterrupt: 

switch proportion rsa

In [66]:
plot_rsa_correlation_by_time(rsa_results, 'switchProportion', rois, colors)

KeyboardInterrupt: 

### stats  
1. for each roi, for each neural rdm (time window), compute correlation between a randomly shuffled model rdm and neural rdm. Do this a lot of times (10,000 for now).
2. Find proportion of random correlations that are greater than the actual correlation between the model rdm and the neural rdm. That's the p-value.
3. After getting p-values for all time windows, find clusters in time where p<0.05. Get the lengths of all clusters, and if a cluster isn't in the top 5% of cluster lengths, it's not significant. Keep significant clusters. Not sure if this is  best way to do this. Seems wrong.


In [ ]:
import numpy as np
from scipy.stats import spearmanr
from numpy.random import shuffle

def permutation_test(neural_rdm, model_rdm, num_permutations=10000):
    actual_correlation, _ = spearmanr(neural_rdm.flatten(), model_rdm.flatten())
    permuted_correlations = []
    for _ in range(num_permutations):
        shuffled_model_rdm = np.random.permutation(model_rdm.flatten())
        permuted_corr, _ = spearmanr(neural_rdm.flatten(), shuffled_model_rdm)
        permuted_correlations.append(permuted_corr)
    
    p_value = np.sum(np.abs(permuted_correlations) >= np.abs(actual_correlation)) / num_permutations
    return actual_correlation, p_value

# Example usage
neural_rdm_example = neural_rdms_by_samples_and_roi['lpfc'][0][1]  # Assume some example RDM
model_rdm_example = model_rdms['congruency']
actual_correlation, p_value = permutation_test(neural_rdm_example, model_rdm_example)
print(f"Actual correlation: {actual_correlation}, P-value: {p_value}")
